### Scrape data from HTML tables into a DataFrame using BeautifulSoup and Pandas
- Author : Nijatullah Mansoor 
- Date   : 08/27/2021

Let's get <a href="https://ca.finance.yahoo.com/quote/TSLA/history?p=TSLA">Tesla Historical</a> data and store in a dataFrame.

Let's import the required library.

In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

In [2]:
# url for the website 
url = 'https://finance.yahoo.com/quote/TSLA/history/'
url1 = 'https://ca.finance.yahoo.com/'

let's get the web page content.

In [3]:
# get the contents of the webpage in text format and store in a variable called data
HEADER = {"User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}

data  = requests.get(url,headers=HEADER)

Let's chekc the response of request.

In [4]:
data

<Response [404]>

Let's create a BeautifulSoup object.

In [5]:
requests.get('https://finance.yahoo.com/quote/TSLA/history/')

<Response [404]>

refer to this to resolve this issue.

https://stackoverflow.com/questions/47506092/python-requests-get-always-get-404

From the response we can see that `<Response [404]>` which represent page not found.

In [142]:
data = requests.get('https://finance.yahoo.com/quote/TSLA/history?period1=1288310400&period2=1630108800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true', headers={'User-Agent': 'Custom'})

Now let's check the page response status.

In [143]:
print(data)

<Response [200]>


Let's create a BeautifulSoup object.

In [126]:
soup = BeautifulSoup(data.content,'html.parser')

In [127]:
table = soup.find('div', id='Col1-1-HistoricalDataTable-Proxy')

Let's chekc how the data are arranged in the website. 

<img src='tesla_website_data_structure.png'>

so we will get the data in the same manager and store in the panda dataFrame.

let's get the `Time Period` of this data.


In [128]:
info = table.find('div',class_="Pt(15px)")

In [129]:
table_data = table.find('table',class_='W(100%) M(0)')

In [130]:
table_header = table_data.find('thead')

In [131]:
columns_name = []

for i in table_header.find_all('span'):
    columns_name.append(i.text)

In [132]:
columns_name

['Date', 'Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume']

Now let's get the actual data from the table. 

In [133]:
table_body = table_data.find('tbody')

In [134]:
date = []
Open = []
high = []
low = []
close = []
adj_close = []
volume = []

# we will loop through every table row 
for tr in table_body.find_all('tr'):
    i=0
    # we will loop through every tow data and get the data from it.
    for td in tr.find_all('td'):
        if i==0:
            date.append(td.find('span').text)
            i=1
        elif i==1:
            Open.append(td.find('span').text)
            i=2
        elif i==2:
            high.append(td.find('span').text)
            i=3
        elif i==3:
            low.append(td.find('span').text)
            i=4
        elif i==4:
            close.append(td.find('span').text)
            i=5
        elif i==5:
            adj_close.append(td.find('span').text)
            i=6
        elif i==6:
            volume.append(td.find('span').text)

Let's now create a dataFrame from this. 

In [136]:
df = pd.DataFrame(columns=columns_name)

In [138]:
df

,Date,Open,High,Low,Close*,Adj Close**,Volume


In [139]:
df['Date'] = date
df['Open'] = Open
df['High'] = high
df['Low'] = low
df['Close*'] = close
df['Adj Close**'] = adj_close
df['Volume'] = volume

In [140]:
df.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Aug 27, 2021",705.00,715.00,702.10,711.92,711.92,"13,762,100"
1,"Aug 26, 2021",708.31,715.40,697.62,701.16,701.16,"13,214,300"
2,"Aug 25, 2021",707.03,716.97,704.00,711.20,711.20,"12,645,600"
3,"Aug 24, 2021",710.68,715.22,702.64,708.49,708.49,"13,083,100"
4,"Aug 23, 2021",685.44,712.13,680.75,706.30,706.30,"20,264,900"


We successfully got the data from the website. 

In [141]:
len(df)

100

Let's covert this to a `.csv` file.

In [144]:
df.to_csv('yahoo_finance_tesla.csv',index=False)